In [1]:
import sys 
import os

In [2]:
src_path = os.path.abspath("../")
if src_path not in sys.path:
    sys.path.append(src_path)

In [3]:
from src.tools.md_class_functions import *
from src.tools.md_class_utility import*
from src.tools.md_class_graphs import *
from src.water_md_class import *

### Set path to your lammps file and initialise trajectory object

In [ ]:
# path towards an ion trajectory with recombination happening
water_path = "../testing/recombination_tester.lammpstrj"

In [ ]:
 #initialise trajectory object by passing path, format and scaling = 0 means not scaled -> will be scaled in __init__
trj = Trajectory(water_path, format="lammpstrj", scaled=0, verbosity="silent")

### Access class atributes of the Trajectory object 

In [ ]:
    # access class atributes which are calculated on initialisation, basic information about the trajectory 
    print("Trajectory first 3 rows")
    print(trj.trajectory[0, :3, :])
    print("box dimensions")
    print(trj.box_dim[0])
    print("box size")
    print(trj.box_size[0])
    print("number of atoms")
    print(trj.n_atoms)
    print("number of timesteps")
    print(trj.n_snapshots)
    print("species split")
    print("s1 = Hydrogen")
    print(trj.s1[0][:3, :])
    print("s2 = Oxygen")
    print(trj.s2[0][:3, :])
    print("did recombine?")
    print(trj.did_recombine)
    print("Recombination Time")
    print(trj.recombination_time)

### Class Methods


In [ ]:
#sets the self.distance attribute
trj.get_ion_distance()

In [ ]:
plot_ion_distance_euc(trj, (11, 6))

In [ ]:
#for further evaluations i just pick one timestep < recombination_time
time_step = 773

In [ ]:
bonding_list, unique_oxygens, ions = trj.get_hydrogen_bonds(timestep=time_step, cutoff=2.9, starting_oh=True)

In [ ]:
plot_hbonds_single(bonding_list, trj.s2[time_step], start="OH", fig_size=(10, 8))

In [ ]:
bonds_H3O, oxygens_H3O, ions =  trj.get_hydrogen_bonds(timestep=time_step, cutoff=2.9, starting_oh=False)
bonds_OH, oxygens_OH, _ =  trj.get_hydrogen_bonds(timestep=time_step, cutoff=2.9, starting_oh=True)

In [ ]:
plot_hbond_network(bonds_OH, bonds_H3O, trj.s2[time_step], ions, fig_size=(10, 8))

In [ ]:
# HB bonds for the entire trajectory
hb_timeseries = get_HB_timeseries(trj)

In [ ]:
# use %matplotlib widget or similar to make interactive
plot_HB_network(hb_timeseries, trj.s2, plot_oxygen=True)

In [ ]:
plot_HB_ratio(hb_timeseries, n_atoms=trj.n_atoms)

In [ ]:
wire_lengths, h_bonds = get_all_wires(trj)

In [ ]:
%matplotlib widget

In [ ]:
plot_HB_wire(h_bonds, trj, plot_hydrogens=True)

In [ ]:
plot_wire_length(wire_lengths)

In [ ]:
ts_bonds, mol_list, ion_ts = get_transition_cations(trj)

In [ ]:
plot_transition_cations(mol_list, ion_ts, trj)

In [ ]:
OO_rdf = trj.get_rdf_rdist(stop=8.0)

In [ ]:
OO_rdf

In [ ]:
HH_rdf = trj.get_rdf_rdist(gr_type="HH")

In [ ]:
plot_rdf(OO_rdf[0], OO_rdf[1])

In [ ]:
plot_rdf(HH_rdf[0], HH_rdf[1], type="HH")

In [ ]:
MSD = trj.get_MSD()

In [ ]:
plot_MSD(MSD)

In [ ]:
diff = trj.get_translational_diffusion(MSD)
diff

### Trajectory Manipulation




In [ ]:
#trjwater.lammpstrj is just a water sim trajectory without ions 
#note: (actually already has ions in it but they dont recombine still same concept)
path_water = "../testing/trjwater.lammpstrj"

In [ ]:
traj_2 = Trajectory(file=path_water, format="lammpstrj", scaled=1, verbosity="loud")

In [ ]:
# generate ion trajectories for ion MD runs
traj_2.get_displace(snapshot=50, distance=0.4, eps=0.05, path="../tutorial_notebook/", num_traj=2)

In [ ]:
#cut out trajectory at timestamp 50
traj_2.cut_snapshot(snapshot=50, path="../tutorial_notebook/")

In [ ]:
# remove 20 atoms from timestap 50
traj_2.remove_atoms(N=20, snap=50, format_out="lammps")

In [ ]:
# group together the molecules and writes it into a ovito readable lammpstrj file
traj_2.group_molecules(path="../tutorial_notebook/")

In [ ]:
data = np.loadtxt("C:\\Users\\Nutzer\\Documents\\GitHub\\MD_Lammps_analysis_class\\tutorial_notebook\\OH_ion_RDF_averaged.csv", delimiter=",")
plt.plot(data[1], data[0])

In [ ]:
data = np.loadtxt("C:\\Users\\Nutzer\\Documents\\GitHub\\MD_Lammps_analysis_class\\tutorial_notebook\\H3O_ion_RDF_averaged.csv", delimiter=",")
plt.plot(data[1], data[0])

In [ ]:
h3o_ids_ts = np.empty((trj.recombination_time, ), dtype=int)
oh_ids_ts = np.empty((trj.recombination_time, ), dtype=int)
for ts in range(trj.recombination_time):
    
    OH_id = None
    H3O_id = None

    # note: find nearest O atom for each H atom
    indexlist_group, _ = trj.get_neighbour_KDT(species_1=trj.s1[ts],
                                                species_2=trj.s2[ts], mode="pbc", snapshot=ts)

    # note: find he  number of  occourence of O atoms for which it is the nearest to an H atom.
    # -> for H2O each O atom will count twice, for each H3O+ each O atom will count 3 times and so on.
    temp = [None] * trj.s2[ts].shape[0]
    for O_atom in range(trj.s2[ts].shape[0]):
        temp[O_atom] = np.append(np.argwhere(indexlist_group == O_atom), O_atom)

    # check how often each O atom counted -> molecules formation  OH- = 1 time H3O+  3 Times  H2O 2 times.
    for ind, _list in enumerate(temp):
        #if len(_list) == 2:
         #   OH_id = _list[-1]
        if len(_list) == 4:
            H3O_id = _list[-1]
            
    h3o_ids_ts[ts] = trj.s2[ts][H3O_id, 0]
    #oh_ids_ts[ts] = trj.s2[ts][OH_id, 0]


jumps = []
diffusion = []
for position_id in range(1, trj.recombination_time):
    if h3o_ids_ts[position_id-1] != h3o_ids_ts[position_id]:
        jumps.append(position_id-1)
    else:
        diffusion.append(position_id-1)
        
        

    


In [ ]:
h3o_ids_ts

In [ ]:
jumps

In [ ]:
diffusion

In [ ]:
def get_diffusion_distance(jumps: [int], diffusion: [int], ion_ids: [int], trj: Trajectory):
    
    coordinates = trj.s2
    temp = []
    diffusion_distances = []
    diffusion_coordinates = []

    previous = diffusion[0]
    intervalls = []
    _temp = []


    for diff_ts in range(1, len(diffusion)):
        #print(previous)
        if (diffusion[diff_ts] - 1 == previous): 
            _temp.append(previous)
            previous = diffusion[diff_ts]
        else:
            _temp.append(previous)
            if len(_temp) > 1:
                intervalls.append(_temp)
            _temp = []
            previous = diffusion[diff_ts]
    print(intervalls)
    for diffusion_int in intervalls:
        for diff in range(len(diffusion_int) -1):
            temp.append(get_distance(coordinates[diffusion_int[diff]][coordinates[diffusion_int[diff]][:, 0]==ion_ids[diffusion_int[diff]], 2:][0],
                                    coordinates[diffusion_int[diff+1]][coordinates[diffusion_int[diff+1]][:, 0]==ion_ids[diffusion_int[diff+1]], 2:][0],
                                    mode="pbc"))
        diffusion_distances.append(sum(temp))
        temp = []
    return diffusion_distances
        

In [ ]:
def get_jump_distances(jumps: [int], ion_ids: [int], trj: Trajectory):
    
    coordinates = trj.s2
    jump_distances = []
    
    for jump_ts in range(len(jumps)):
        jump_distances.append(get_distance(coordinates[jumps[jump_ts]][coordinates[jumps[jump_ts]][:, 0] == ion_ids[jumps[jump_ts]], 2:][0],
                                          coordinates[jumps[jump_ts]-1][coordinates[jumps[jump_ts]-1][:, 0] == ion_ids[jumps[jump_ts]-1], 2:][0],
                                          mode="pbc"))
        
    return jump_distances

In [ ]:
diff_dist = get_diffusion_distance(jumps, diffusion, h3o_ids_ts, trj)
jump_dist = get_jump_distances(jumps, h3o_ids_ts, trj)

In [ ]:
len(jump_dist)

In [ ]:
len(diff_dist)

In [ ]:
jump_dist

In [ ]:
import struct
print(struct.calcsize("P") * 8)

In [ ]:
path = "Z://cluster_runs//n_608//NN_charged_test//results//charged_system//charged_run_0//trjwater.lammpstrj"

In [ ]:
trj = Trajectory(file=path)

In [ ]:
trj.did_recombine

In [ ]:
diffusion, jumps, h3_ids_ts = diffusion_timestep_tracing(trj)

In [ ]:
diff_dist = get_diffusion_distance(diffusion, h3_ids_ts, trj)

In [ ]:
jump_dist = get_jump_distances(jumps, h3_ids_ts, trj)

In [ ]:
len(jump_dist)

In [ ]:
len(diff_dist)

In [ ]:

def autocorrelation_function(distances):
    """
    Calculate the autocorrelation function of a list of distances.
    
    Parameters:
    distances (list or np.ndarray): List or array of distance values (diffusion or jump distances).
    
    Returns:
    np.ndarray: Autocorrelation function for the input distance list.
    """
    distances = np.array(distances)
    N = len(distances)
    mean_distance = np.mean(distances)
    
    acf = np.zeros(N)
    
    for tau in range(N):
        acf[tau] = np.mean((distances[:N - tau] - mean_distance) * (distances[tau:] - mean_distance))
    
    return acf / acf[0]  # Normalize by the value at tau=0 (to get a normalized ACF)


In [ ]:
# Compute autocorrelation functions
acf_diffusion = autocorrelation_function(jump_dist)
acf_jump = autocorrelation_function(diff_dist)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(acf_diffusion, label='Diffusion ACF', color='blue')
plt.plot(acf_jump, label='Jump ACF', color='red')
plt.xlabel('Lag (τ)')
plt.ylabel('Autocorrelation')
plt.legend()
plt.title('Autocorrelation Function of Diffusion and Jump Distances')
plt.grid(True)
plt.show()

In [ ]:
np.savetxt(r"Z:\\cluster_runs\\n_608\\NN_charged_test\\results\\charged_system\\long_run\\charged_run_0\\jump_dist.csv", jump_dist)

In [ ]:
# Compute autocorrelation functions
acf_diffusion = autocorrelation_function(jump_dist)
acf_jump = autocorrelation_function(diff_dist)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(acf_diffusion, label='Diffusion ACF', color='blue')
plt.plot(acf_jump, label='Jump ACF', color='red')
plt.xlabel('Lag (τ)')
plt.ylabel('Autocorrelation')
plt.legend()
plt.title('Autocorrelation Function of Diffusion and Jump Distances')
plt.grid(True)
plt.show()

In [ ]:
def cross_autocorrelation(x, y):
    x = np.asarray(x) - np.mean(x)
    y = np.asarray(y) - np.mean(y)
    corr = np.correlate(x, y, mode='full')
    corr = corr[corr.size // 2:]
    norm = np.sqrt(np.dot(x, x) * np.dot(y, y))  # normalization by magnitude
    return corr / norm

In [ ]:
# Compute autocorrelation functions
acf_diffusion = cross_autocorrelation(jump_dist, diff_dist)
#acf_jump = autocorrelation_function(diff_dist)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(acf_diffusion, label='Diffusion ACF', color='blue')
#plt.plot(acf_jump, label='Jump ACF', color='red')
plt.xlabel('Lag (τ)')
plt.ylabel('Autocorrelation')
plt.legend()
plt.title('Cross Autocorrelation Function of Diffusion and Jump Distances')
plt.grid(True)
plt.show()

In [ ]:
# Compute autocorrelation functions
acf_diffusion = cross_autocorrelation(diff_dist, diff_dist)
#acf_jump = autocorrelation_function(diff_dist)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(acf_diffusion, label='Diffusion ACF', color='blue')
#plt.plot(acf_jump, label='Jump ACF', color='red')
plt.xlabel('Lag (τ)')
plt.ylabel('Autocorrelation')
plt.legend()
plt.title('Cross Autocorrelation Function of Diffusion Distances')
plt.grid(True)
plt.show()

In [ ]:
# Compute autocorrelation functions
acf_diffusion = cross_autocorrelation(jump_dist, jump_dist)
#acf_jump = autocorrelation_function(diff_dist)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(acf_diffusion, label='Diffusion ACF', color='blue')
#plt.plot(acf_jump, label='Jump ACF', color='red')
plt.xlabel('Lag (τ)')
plt.ylabel('Autocorrelation')
plt.legend()
plt.title('Cross Autocorrelation Function of Jump Distances')
plt.grid(True)
plt.show()

In [ ]:
jump_data = np.genfromtxt(r"C:\Users\karim\Documents\GitRepo\MD_Lammps_analysis\testing\jump_dist.csv")
diff_data = np.genfromtxt(r"C:\Users\karim\Documents\GitRepo\MD_Lammps_analysis\testing\diff_dist.csv")

In [ ]:
plt.hist(jump_data, bins=20)
plt.show()

In [ ]:
bins = np.linspace(0.0, 0.3, 100)
plt.hist(diff_dist, bins=bins)
plt.title("Histogram of Diffusiondistances")
plt.show()

In [ ]:
bins = np.linspace(0.0, 0.0005, 100)
plt.hist(jump_dist, bins=bins)
plt.title("Histogram of Jumpdistances")
plt.show()

In [ ]:
bins = np.linspace(0.0, 0.1, 50)
plt.hist(jump_dist, bins=bins)
plt.title("Histogram of Jumpdistances - long jumps")
plt.show()

In [7]:
import time

In [5]:
file=r"Z:\\cluster_runs\\n_608\\NN_charged_test\\results\\charged_system\\long_run\\charged_run_0\\trjwater.lammpstrj"
file_test = r"Z:\\cluster_runs\\n_608\\NN_charged_test\\results\\charged_system\\charged_run_0\\trjwater.lammpstrj"
save = r"Z:\\cluster_runs\\n_608\\NN_charged_test\\results\\charged_system\\long_run\\charged_run_0\\trjwater.npz"

In [ ]:
start_time = time.time()
trj = Trajectory(file=file_test, scaled=1, batch=True)
end_time = time.time()

Processed 1000 snapshots so far...
Processed 2000 snapshots so far...
Processed 3000 snapshots so far...
Processed 4000 snapshots so far...
Processed 5000 snapshots so far...
Processed 6000 snapshots so far...
Processed 7000 snapshots so far...
Processed 8000 snapshots so far...
Processed 9000 snapshots so far...
Processed 10000 snapshots so far...
Processed 11000 snapshots so far...
Processed 12000 snapshots so far...
Processed 13000 snapshots so far...
Processed 14000 snapshots so far...
Processed 15000 snapshots so far...
Processed 16000 snapshots so far...
Processed 17000 snapshots so far...
Processed 18000 snapshots so far...
Processed 19000 snapshots so far...
Processed 20000 snapshots so far...


In [8]:
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time / 60 :.4f} Minutes")

Elapsed time: 174.3051 Minutes


In [ ]:
trj.trajectory[0, 0, :]

In [ ]:
np.array_equal(trj_2.trajectory, trj.trajectory)

In [6]:
trj = Trajectory(file=file, format="lammpstrj", batch=True, scaled=0)

Processed 1000 snapshots so far...
Processed 2000 snapshots so far...
Processed 3000 snapshots so far...
Processed 4000 snapshots so far...
Processed 5000 snapshots so far...
Processed 6000 snapshots so far...
Processed 7000 snapshots so far...
Processed 8000 snapshots so far...
Processed 9000 snapshots so far...
Processed 10000 snapshots so far...
Processed 11000 snapshots so far...
Processed 12000 snapshots so far...
Processed 13000 snapshots so far...
Processed 14000 snapshots so far...
Processed 15000 snapshots so far...
Processed 16000 snapshots so far...
Processed 17000 snapshots so far...
Processed 18000 snapshots so far...
Processed 19000 snapshots so far...
Processed 20000 snapshots so far...
Processed 21000 snapshots so far...
Processed 22000 snapshots so far...
Processed 23000 snapshots so far...
Processed 24000 snapshots so far...
Processed 25000 snapshots so far...
Processed 26000 snapshots so far...
Processed 27000 snapshots so far...
Processed 28000 snapshots so far...
P

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Nutzer\anaconda3\envs\master_thesis\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Nutzer\AppData\Local\Temp\ipykernel_23224\2783189507.py", line 1, in <module>
    trj = Trajectory(file=file, format="lammpstrj", batch=True, scaled=0)
  File "C:\Users\Nutzer\Documents\GitHub\MD_Lammps_analysis_class\src\water_md_class.py", line 45, in __init__
    batch_size)
  File "C:\Users\Nutzer\Documents\GitHub\MD_Lammps_analysis_class\src\water_md_class.py", line 129, in lammpstrj_to_np_batchwise
    atom_data = atom_data[:, [0, 1, 2, 3, 4]]  # id, species, x, y, z
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 71.3 KiB for an array with shape (5, 1825) and data type float64

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Nutzer\anaconda3\envs\master_thesis\lib

TypeError: object of type 'NoneType' has no len()

In [ ]:
msd_diff = []
msd_jump = []
cross_term = []
msd_total = []

for t in range(len(r_diff)):
    dr_diff = r_diff[t] - r_diff[0]
    dr_jump = r_jump[t] - r_jump[0]
    
    diff_sq = np.dot(dr_diff, dr_diff)
    jump_sq = np.dot(dr_jump, dr_jump)
    cross = np.dot(dr_diff, dr_jump)
    
    msd_diff.append(diff_sq)
    msd_jump.append(jump_sq)
    cross_term.append(cross)
    msd_total.append(diff_sq + jump_sq + 2 * cross)

In [ ]:
diffusion